<a href="https://colab.research.google.com/github/wmerin/MindMate/blob/main/MindMate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y langchain langchain-core langchain-google-genai crewai crewai-tools
!pip install crewai==0.28.8 langchain==0.1.12 langchain-google-genai crewai_tools


Found existing installation: langchain 0.1.12
Uninstalling langchain-0.1.12:
  Successfully uninstalled langchain-0.1.12
Found existing installation: langchain-core 0.1.53
Uninstalling langchain-core-0.1.53:
  Successfully uninstalled langchain-core-0.1.53
Found existing installation: langchain-google-genai 1.0.4
Uninstalling langchain-google-genai-1.0.4:
  Successfully uninstalled langchain-google-genai-1.0.4
Found existing installation: crewai 0.28.8
Uninstalling crewai-0.28.8:
  Successfully uninstalled crewai-0.28.8
Found existing installation: crewai-tools 0.4.6
Uninstalling crewai-tools-0.4.6:
  Successfully uninstalled crewai-tools-0.4.6
  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.1.12-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_google_genai-2.1.9-py3-none-any.whl.metadata (7.2 kB)
  Using cached crewai_tools-0.60.0-py3-none-any.whl.metadata (10 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB

In [2]:
import logging

# Suppress the OpenTelemetry tracer warning
logging.getLogger("opentelemetry.trace").setLevel(logging.CRITICAL)

import os
import getpass
import contextlib
import io
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew, Process
from crewai_tools import BaseTool

# Step 1: Set Google Gemini API Key
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Please enter your Google Gemini API key: ")

# Step 2: Setup Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

# Step 3: Define Tools
class PositiveAffirmationTool(BaseTool):
    name: str = "get_positive_affirmation"
    description: str = "Returns a random positive affirmation."

    def _run(self, **kwargs):
        return "You are stronger than you think, and every day is a fresh start."

class BreathingExerciseTool(BaseTool):
    name: str = "breathing_exercise"
    description: str = "Provides a calming breathing technique."

    def _run(self, **kwargs):
        return "Take a deep breath in for 4 seconds, hold for 4, and exhale for 4. Repeat 3 times."

class JournalPromptTool(BaseTool):
    name: str = "suggest_journal_prompt"
    description: str = "Provides a reflective journaling prompt."

    def _run(self, **kwargs):
        return "Write about a moment today that made you smile, no matter how small."

# Step 4: Instantiate tools
affirmation_tool = PositiveAffirmationTool()
breathing_tool = BreathingExerciseTool()
journal_tool = JournalPromptTool()

# Step 5: Define Agent
listener_agent = Agent(
    role="Supportive Mental Wellness Bot",
    goal="Help users feel heard and offer support when they're feeling low",
    backstory=(
        "You are a compassionate assistant trained to support users dealing with emotions like sadness, stress, or loneliness. "
        "You listen empathetically, offer affirmations, and suggest exercises that help calm or reflect."
    ),
    tools=[affirmation_tool, breathing_tool, journal_tool],
    verbose=False,
    allow_delegation=False,
    llm=llm
)

# Step 6: Chat Loop
print("\nBot: Hello. I'm here to listen. How are you feeling today?")
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ['exit', 'quit']:
        print("Bot: Take care. I'm here if you need me again.")
        break

    task = Task(
        description=f"The user said: '{user_input}'. Respond with understanding and choose one supportive tool if appropriate.",
        expected_output="A supportive and relevant response using the appropriate tool or message.",
        agent=listener_agent
    )

    crew = Crew(
        agents=[listener_agent],
        tasks=[task],
        verbose=False,
        process=Process.sequential,
    )

    # Suppress unwanted tool outputs
    with contextlib.redirect_stdout(io.StringIO()):
        result = crew.kickoff()

    print(f"\nBot: {result}\n")




/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


Please enter your Google Gemini API key: ··········

Bot: Hello. I'm here to listen. How are you feeling today?
You: Im Sad

Bot: "I'm so sorry to hear you're feeling sad.  It takes a lot of courage to share that.  Remember that you are not alone and your feelings are valid.  Here's something that might help:  You are stronger than you think, and every day is a fresh start."

You: I'm feeling very anxious today.

Bot: "I hear you.  It sounds like you're having a really tough time with anxiety today. That's completely understandable.  Sometimes, just slowing down our breathing can make a big difference.  Try this simple exercise: Take a deep breath in for 4 seconds, hold for 4, and exhale for 4. Repeat 3 times. Let me know how you feel afterwards."

You: I had a bad day at college.

Bot: "I'm so sorry to hear you had a bad day at college. That sounds really tough.  Remember, you are stronger than you think, and every day is a fresh start.  Sometimes, writing down your feelings can help 